In [15]:
import pandas as pd
from apyori import apriori
from mlxtend.frequent_patterns import fpgrowth
from mlxtend.frequent_patterns import association_rules
from mlxtend.preprocessing import TransactionEncoder

In [16]:
df = pd.read_csv('https://github.com/FlavioAAMotta/didatico/raw/main/CD/2024/Associacao/compras.csv')

In [17]:
df.head()

,cliente,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17
0,JÚLIO DEMARQUE,limão,peito de peru,manga,cerveja,picanha,carvão,sal,linguiça,mel,geleia,cereal,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,MAYCON SOARES,salsicha,cerveja,picanha,catchup,geleia,cebola,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,WEMERSON NETO,queijo,frutas,chá,alface,granola,feijão,farinha de trigo,suco de laranja,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,JOÃO DEMARQUE,pimenta,sardinha enlatada,barra de cereal,molho de tomate,gelatina,vinho branco,pão,tempero pronto,vinho tinto,presunto,arroz,feijão,NaN,NaN,NaN,NaN,NaN,NaN
4,MAYCON SANTOS,detergente,filé de frango,arroz,feijão,limão,papel higiênico,presunto,farinha de trigo,salada,alface,cenoura,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [18]:
#dropar coluna dos nomes, pra usar apriori nao precisa saber os nomes
df = df.drop('cliente', axis=1)
df.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17
0,limão,peito de peru,manga,cerveja,picanha,carvão,sal,linguiça,mel,geleia,cereal,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,salsicha,cerveja,picanha,catchup,geleia,cebola,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,queijo,frutas,chá,alface,granola,feijão,farinha de trigo,suco de laranja,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,pimenta,sardinha enlatada,barra de cereal,molho de tomate,gelatina,vinho branco,pão,tempero pronto,vinho tinto,presunto,arroz,feijão,NaN,NaN,NaN,NaN,NaN,NaN
4,detergente,filé de frango,arroz,feijão,limão,papel higiênico,presunto,farinha de trigo,salada,alface,cenoura,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [19]:
transactions = []
for i in range(0,df.shape[0]):
    transactions.append([str(df.values[i,j])for j in range(0,df.shape[1])])

In [20]:
#definindo o suporte mínimo como 0.02
#estamos interessados apenas em conjuntos de itens que aparecem em pelo menos 2% das transações. 
regras = apriori(transactions=transactions, min_support=0.05, min_confidence=0.2, min_lift=5, min_length=2, max_length=2)

In [21]:
resultados = list(regras)
resultados

[RelationRecord(items=frozenset({'arroz', 'feijão'}), support=0.0544, ordered_statistics=[OrderedStatistic(items_base=frozenset({'arroz'}), items_add=frozenset({'feijão'}), confidence=0.6384976525821596, lift=6.223174001775435), OrderedStatistic(items_base=frozenset({'feijão'}), items_add=frozenset({'arroz'}), confidence=0.530214424951267, lift=6.223174001775435)])]

In [22]:
df_compras_lista = df.apply(lambda x: x.dropna().tolist(), axis=1).tolist()

te = TransactionEncoder()
te_ary = te.fit(df_compras_lista).transform(df_compras_lista)

# Converte o array resultante em um DataFrame pandas
df_onehot = pd.DataFrame(te_ary, columns=te.columns_)

# Agora você pode aplicar o fpgrowth no DataFrame one-hot
frequent_itemsets = fpgrowth(df_onehot, min_support=0.003,use_colnames=True)
rules = association_rules(frequent_itemsets, metric="lift", min_threshold=3)

In [23]:
rules.size

14820

In [24]:
# imprimindo as regras mais importantes ordenadas por lift
rules = rules.sort_values(by='lift', ascending=False)
rules.head(10)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
712,"(catchup, hambúrguer)","(batata frita, cerveja, mostarda)",0.0145,0.0056,0.0041,0.282759,50.492611,0.004019,1.386423,0.994617
697,"(batata frita, cerveja, mostarda)","(catchup, hambúrguer)",0.0056,0.0145,0.0041,0.732143,50.492611,0.004019,3.679200,0.985715
713,"(mostarda, hambúrguer)","(batata frita, catchup, cerveja)",0.0110,0.0074,0.0041,0.372727,50.368550,0.004019,1.582406,0.991048
696,"(batata frita, catchup, cerveja)","(mostarda, hambúrguer)",0.0074,0.0110,0.0041,0.554054,50.368550,0.004019,2.217758,0.987453
1064,"(azeite, salada)","(cenoura, vinagre, tomate)",0.0151,0.0054,0.0040,0.264901,49.055678,0.003918,1.353014,0.994634
1053,"(cenoura, vinagre, tomate)","(azeite, salada)",0.0054,0.0151,0.0040,0.740741,49.055678,0.003918,3.798900,0.984934
1268,"(vinagre, alface, salada)","(tomate, azeite)",0.0078,0.0111,0.0042,0.538462,48.510049,0.004113,2.142617,0.987085
1273,"(tomate, azeite)","(vinagre, alface, salada)",0.0111,0.0078,0.0042,0.378378,48.510049,0.004113,1.596148,0.990379
1065,"(tomate, azeite)","(cenoura, vinagre, salada)",0.0111,0.0076,0.0040,0.360360,47.415837,0.003916,1.551499,0.989898
1052,"(cenoura, vinagre, salada)","(tomate, azeite)",0.0076,0.0111,0.0040,0.526316,47.415837,0.003916,2.087678,0.986407
